In [2]:
import random
from hashlib import sha256

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

def schnorr_sign(message, g, p, s):
    # 选择随机数k
    k = random.randint(1, p-2)
    # 计算r = g^k mod p
    r = pow(g, k, p)
    # 将消息和r的值哈希，然后与私钥s结合产生签名
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    x = (k - s * hash_value) % (p-1)
    return r, x

def schnorr_verify(message, g, p, v, r, x):
    # 重新计算r的哈希值
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    # 使用公钥v和签名x、r计算验证值
    rv = (pow(g, x, p) * pow(v, hash_value, p)) % p
    return rv == r

# 设置大素数p和其原根g
p = 1000000007  # 一个大素数
g = 5  # p的原根

# 生成密钥
s, v = generate_keys(g, p)

# 签名消息
message = "Hello, Schnorr!"
r, x = schnorr_sign(message, g, p, s)
print(f"Signature: (r={r}, x={x})")

# 验证签名
valid = schnorr_verify(message, g, p, v, r, x)
print(f"Signature valid: {valid}")


Signature: (r=216489087, x=252977741)
Signature valid: True


In [93]:
# 设置大素数p和其原根g
p = 1000000007  # 一个大素数
g = 5  # p的原根

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

r, R = generate_keys(g, p)
e3 = int(sha256((str(msg) + str(Y) + str(pow(g, r, p))).encode()).hexdigest(), 16)
print('e3:', e3)

_, random_s3 = generate_keys(g, p)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
e1 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
print('e1:', e1)

_, random_s1 = generate_keys(g, p)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
print('e2:', e2)

s2 = (r - x2 * e2)
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3
}

print('signature process total result:')
display(total_res)

msg: Hello, Ring Signature
pubkeys: [841255009, 649069732, 23212975]
Y: 84125500964906973223212975
e3: 88649382698444855426563625103162528956486330126166402314463816367987290940481
random_s3: 604894976
e1: 34794226520967787024232960355194709198779969750736876072503207928827879411708
random_s1: 515058661
e2: 93213463199084036771028118201965710417601356352333856440920382676283643110020
s2: -34414567584573853633381503651516792396434764005965757128202051794008347897873899538891
signature process total result:


{'e1': 34794226520967787024232960355194709198779969750736876072503207928827879411708,
 's1': 515058661,
 's2': -34414567584573853633381503651516792396434764005965757128202051794008347897873899538891,
 's3': 604894976}

In [94]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)

print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)

34794226520967787024232960355194709198779969750736876072503207928827879411708
93213463199084036771028118201965710417601356352333856440920382676283643110020
88649382698444855426563625103162528956486330126166402314463816367987290940481
True


In [79]:
(pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p

135642210

In [80]:
pow(g, r, p)

135642210